In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import os
from tkinter import *

In [43]:
import sys
print (sys.version)

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [42]:
from tkinter import *
from tkinter import filedialog as fd
import demoji
demoji.download_codes()

def find_replace_emo(text):
    all_emoji = demoji.findall(text)
    new_text = 'Emojis: \n'
    if len(all_emoji)>0:
        for i,j in enumerate(all_emoji):
            item = all_emoji[j]
            new_text = new_text+str(i+1)+' '+all_emoji[j]+'\n'
            text = text.replace(j,'_')
            text = text+'\n\n'+new_text
    else:
        text = text+'\n\n'+ new_text+'\n None'
    return text

def next_review():
    global review_df
    global rev_index
    if (var_sent.get()) and (var_emp.get()):
        review_df['sentiment'].loc[rev_index[window._my_hidden_value]] = var_sent.get()
        review_df['reviewer'].loc[rev_index[window._my_hidden_value]] = var_emp.get()
        window._my_hidden_value = window._my_hidden_value+1
        message_1.config(text='')
        txtfld.delete(1.0,END)
        text = review_df['review_EN'].loc[rev_index[window._my_hidden_value]]
        text = find_replace_emo(text)
        txtfld.insert(END, text)
        var_sent.set('')
        var_emp.set('')
        

        remaining = len(review_df[review_df['reviewer'].isnull()].index)
        message_ld2.config(text=str(remaining)+' ('+str(np.round(100*remaining/review_df.shape[0],2))+"%)",
                       font=("Avenir", 12,'bold'))
        
    else:
        message_1.config(text='Message: Please, complete the annotation of current review before going to the Next',
                     fg='red',font=("Avenir", 12,'bold'))
        
def save_annot():
    global review_df
    global filename
    message_1.config(text='')
    review_df.reset_index(inplace = True)
    review_df.to_csv(filename)
    message_2.config(text='Message: Annotations have been saved',
                 fg='green',font=("Avenir", 12,'bold'))
    
    
def load_annot():
    global review_df
    global rev_index
    global filename
    
    filetypes = (
        ('text files', '*.csv'),
    )

    filename = fd.askopenfilename(
        title='Open a file',
        initialdir='./Data/',
        filetypes=filetypes)
#     print(filename)
    review_df = pd.read_csv(filename,index_col = 0)
    #print(review_df.head())
    review_df.set_index('index',inplace = True)
    mask = review_df['reviewer'].isnull()

    rev_index = review_df[mask].index
#     print(mask)
    text = review_df['review_EN'].loc[rev_index[window._my_hidden_value]]
    text = find_replace_emo(text)
    txtfld.insert(END, text)
    
    remaining = len(review_df[review_df['reviewer'].isnull()].index)
    
    message_ld0.config(text=filename.split('/')[-1],font=("Avenir", 10),fg='gray')
    message_ld1.config(text='Left: ',font=("Avenir", 12))
    remaining = len(review_df[review_df['reviewer'].isnull()].index)
    message_ld2.config(text=str(remaining)+' ('+str(np.round(100*remaining/review_df.shape[0],2))+"%)",
                       font=("Avenir", 12,'bold'))



        
###################
x_all = 50
window=Tk()
window.geometry("600x600+10+10")
# p1 = PhotoImage(file = './figures/bird.png')
# window.iconbitmap('./figures/favicon.ico')
window.title('© SAAC 2022  - TOM in Logistics: Annotation GUI - ')
# Setting icon of master window
# window.iconphoto(False, p1)
# window.iconbitmap('./figures/SAAClogo100.png')
window.resizable(False, False) 
window._my_hidden_value = 0

# Create a scrollbar
scrollbar = Scrollbar(window)
scrollbar.pack( side = RIGHT, fill = Y )
# title
lbl=Label(window, text="Reviews Annotation", fg='black', font=("Avenir", 16,'bold'))
lbl.place(x=10, y=10)


k = 50


Load_bt = Button(window, text ="Load Annotation File", command = load_annot, 
                 font=("Avenir", 10,'bold'),bg='red', fg='black', padx=5, pady = 3)
Load_bt.place(x = 10, y = k)


message_ld0 = Label(window, text ="")
message_ld0.place(x = 150, y = k)

message_ld1 = Label(window, text ="")
message_ld1.place(x = 320, y = k)

message_ld2 = Label(window, text ="")
message_ld2.place(x = 350, y = k)

k = k+50
lbl=Label(window, text="Read the review (scroll text to see more)", fg='black', font=("Avenir", 12,'bold'))
lbl.place(x=x_all, y=k)


# Create text widget and specify size.
txtfld = Text(window, height = 5, width = 60, wrap="word", bg = "lightgray", 
              yscrollcommand=scrollbar.set,padx = 20, pady = 20, font = ("Courier", 12))
# 
# txtfld.insert(INSERT, review_df.iloc[0])

# Insert text into the text widget

txtfld.pack( side = LEFT, fill = BOTH )

scrollbar.config(command=txtfld.yview)
scrollbar.pack(fill=Y)
# txtfld.see("end")
txtfld.place(x=x_all, y=k+30)

    

k = k+120

lbl=Label(window, text="Indicate if you think it is a positive/neutral/negative review", fg='black', 
          font=("Avenir", 12,'bold'))
lbl.place(x=x_all, y=k+50)

var_sent = StringVar()
Good = Radiobutton(window, text="Positive", variable=var_sent, value='Positive',font = ("Courier", 12))
Good.place(x=x_all, y=k+100)
# Good.pack( anchor = window)
Neutral = Radiobutton(window, text="Neural/Unclear", variable=var_sent, value='Neural',font = ("Courier", 12))
Neutral.place(x=x_all+100, y=k+100)
# Neutral.pack( anchor = window)
Bad = Radiobutton(window, text="Negative", variable=var_sent, value='Negative',font = ("Courier", 12))
Bad.place(x=x_all+250, y=k+100)
# Bad.pack( anchor = window)


lbl=Label(window, text="Indicate if you think the review was written by client/employee/unknown", 
          fg='black', font=("Avenir", 12,'bold'))
lbl.place(x=x_all, y=k+150)

var_emp = StringVar()
Client = Radiobutton(window, text="Client", variable=var_emp, value='Client',font = ("Courier", 12))
Client.place(x=x_all, y=k+200)
Employee = Radiobutton(window, text="Employee", variable=var_emp, value='Employee',font = ("Courier", 12))
Employee.place(x=x_all+100, y=k+200)
Unknown = Radiobutton(window, text="Undefined", variable=var_emp, value='Undefined',font = ("Courier", 12))
Unknown.place(x=x_all+250, y=k+200)


next_bt = Button(window, text ="Go to Next Review >>>", command = next_review, font=("Avenir", 10,'bold'), 
                 padx=5, pady = 3)
next_bt.place(x = 10, y = k+250)
# sub_bt = Button(top, text ="Summit your Annotation", command = helloCallBack)


message_1 = Label(window)
message_1.place(x = 10, y = k+290)


save_bt = Button(window, text ="Save Annotations", command = save_annot, font=("Avenir", 10,'bold'), 
                 padx=5, pady = 3)
save_bt.place(x = 10, y = k+320)


message_2 = Label(window)
message_2.place(x = 150, y = k+320)

window.mainloop()

... OK (Got response in 0.39 seconds)
Writing emoji data to /Users/giu/.demoji/codes.json ...
... OK
